In [74]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image


from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler

In [75]:
class CNN_Model(nn.Module):
    #列出需要哪些層
    def __init__(self):
        super(CNN_Model, self).__init__()
        # Convolution 1 , input_shape=(3,224,224)
        self.cnn1 = nn.Conv2d(3, 16, kernel_size=5, stride=1) 
        self.relu1 = nn.ReLU(inplace=True) 
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        # (500-5+1)/(1+1) = 248
        # Convolution 2, input_shape=(16,248,248)    
        self.cnn2 = nn.Conv2d(16,8, kernel_size=11, stride=1) 
        self.relu2 = nn.ReLU(inplace=True) 
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        # (248-5+1)/(1+1) = 122
        # Fully connected 1 ,#input_shape=(8*50*50)
        self.fc = nn.Linear(8 * 50 * 50, 2)     
    #列出forward的路徑，將init列出的層代入
    def forward(self, x):
        out = self.cnn1(x) 
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1) 
        out = self.fc(out) 
        return out   

In [109]:
transform = transforms.Compose(
    [transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))])

class product_dataset(Dataset):
    def __init__(self, csv_file, phase, transform = transform):
        self.data      = pd.read_csv(csv_file)
        self.transform = transform
        self.phase     = phase
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]
        origin_img = Image.open(img_path).convert('RGB')
        img = self.transform(origin_img)
        label = self.data.iloc[idx, 1]
        
        sample = {'I': img, 'L': label}
        return sample
        
        
train_set = product_dataset(csv_file = "./dataset/train/train_label.csv", phase='train', transform = transform)
test_set = product_dataset(csv_file = "./dataset/test/test_label.csv", phase='test', transform = transform)
# Create train/valid loaders
train_loader = torch.utils.data.DataLoader(dataset = train_set, batch_size = 10, shuffle = True, num_workers = 0)
test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = 10, shuffle = True, num_workers = 0)
dataiter = iter(test_loader)

classes = ("non_popular", "popular")

In [110]:
total_epochs = 1
n_class = 2
step_size  = 50
gamma      = 0.5
model = CNN_Model()
criterion = nn.CrossEntropyLoss()    
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
scheduler = lr_scheduler.StepLR(optimizer, step_size = step_size, gamma = gamma)  

In [115]:
def train():
    for epoch in range(total_epochs):
        running_loss = 0.0
        model.train()
        scheduler.step()
        for iter, batch in enumerate(train_loader):
            optimizer.zero_grad()
            inputs, labels = Variable(batch['I']), Variable(batch['L'])
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch+1, iter, loss.item()))

        model_path = "./model/epoch_" + str(epoch)        
        torch.save(model.state_dict(), model_path + '.pkl')
        # test(epoch)
    print('Finished Training')
            

In [118]:
train()

epoch1, iter0, loss: 0.6710184216499329
epoch1, iter10, loss: 0.5463494658470154
epoch1, iter20, loss: 0.5902472734451294
epoch1, iter30, loss: 0.6465815305709839
epoch1, iter40, loss: 0.5888835191726685
epoch1, iter50, loss: 0.444852739572525
epoch1, iter60, loss: 0.42926734685897827
epoch1, iter70, loss: 0.6700798273086548
epoch1, iter80, loss: 0.6868084073066711
epoch1, iter90, loss: 0.5105764269828796
epoch1, iter100, loss: 0.4647194743156433
epoch1, iter110, loss: 0.5473817586898804
epoch1, iter120, loss: 0.6025976538658142
epoch1, iter130, loss: 0.7460258603096008
epoch1, iter140, loss: 0.5423699617385864
epoch1, iter150, loss: 0.42896491289138794
epoch1, iter160, loss: 0.5068690180778503
epoch1, iter170, loss: 0.4848204255104065
epoch1, iter180, loss: 0.5513542890548706
epoch1, iter190, loss: 0.6129274368286133
epoch1, iter200, loss: 0.616765558719635
epoch1, iter210, loss: 0.6097763180732727
epoch1, iter220, loss: 0.6036956310272217
epoch1, iter230, loss: 0.5283913016319275


KeyboardInterrupt: 

In [11]:
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in test_loader:
        inputs, labels = Variable(batch['X']), Variable(batch['Y'])
        output = model(inputs)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

KeyError: 211